In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def fetch_relevant_data (url, keywords):
    try:
        #Fetch page Content
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        #Extract text and search keywords
        relevant_data = []
        for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'a']):
            text_content = element.get_text(strip=True)
            if any(keyword.lower() in text_content.lower() for keyword in keywords):
                relevant_data.append(text_content)

        #Return relevant selections of text
        if relevant_data:
            return relevant_data
        else:
            return "No relevant data found for the specified keywords."
        
    except requests.RequestException as e:
        return f"An error occured: {e}"

In [ ]:
# Usage example
url = "https://www.denvergov.org/Government/Agencies-Departments-Offices/Agencies-Departments-Offices-Directory/Denver-City-Council/Council-Members-Websites-Info/District-4"
keywords = ["budget", "housing", "public safety"]
print(fetch_relevant_data(url, keywords))

In [ ]:
def fetch_and_display_summaries_with_links(topics: str):
    url = "https://www.denvergov.org/Government/Agencies-Departments-Offices/Agencies-Departments-Offices-Directory/Denver-City-Council/Council-Members-Websites-Info/District-4"
    keywords = [topic.strip() for topic in topics.split(',')]
    try:
        # Fetch and parse the page content
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find relevant sections and generate summaries
        relevant_articles = {}
        for keyword in keywords:
            relevant_articles[keyword] = []
            for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'a']):
                if keyword.lower() in element.get_text(strip=True).lower():
                    # Capture the parent section for full context
                    section = element.find_parent()
                    full_text = section.get_text(strip=True, separator="\n") if section else element.get_text(strip=True)
                    
                    # Generate a short summary (first 1-2 sentences)
                    summary = '. '.join(full_text.split('. ')[:2]) + '...'
                    
                    # Add link if it's a URL element
                    link = url if not element.name == 'a' else element.get('href', url)
                    
                    # Avoid duplicates
                    if summary not in [item['summary'] for item in relevant_articles[keyword]]:
                        relevant_articles[keyword].append({'summary': summary, 'link': link})
        
        # Format the output with summaries and links
        display_output = "### Summaries with Links to Full Articles:\n\n"
        for keyword, articles in relevant_articles.items():
            display_output += f"**Keyword: {keyword.capitalize()}**\n\n"
            for i, article in enumerate(articles, 1):
                display_output += f"{i}. {article['summary']} "
                display_output += f"[Read More]({article['link']})\n\n"
                display_output += "---\n"  # Separator between summaries
            
            if not articles:
                display_output += f"No articles found for **{keyword}**.\n\n"

        return display_output

    except requests.RequestException as e:
        return f"An error occurred: {e}"

In [ ]:
print(fetch_and_display_summaries_with_links("Police, Homeless"))

In [ ]:
import urllib.parse
def fetch_and_display_search_results(base_url, keywords):
    def get_relevant_content(search_url):
        # Make a search request to the search URL
        search_response = requests.get(search_url)
        search_response.raise_for_status()
        search_soup = BeautifulSoup(search_response.text, 'html.parser')
        
        # Extract relevant sections
        relevant_articles = []
        for element in search_soup.find_all(['p', 'h1', 'h2', 'h3', 'a']):
            text_content = element.get_text(strip=True)
            # Capture the parent section for full context
            section = element.find_parent()
            full_text = section.get_text(strip=True, separator="\n") if section else text_content
            
            # Generate a short summary (first 1-2 sentences)
            summary = '. '.join(full_text.split('. ')[:2]) + '...'
            # Link from the element or the main search URL
            link = search_url if not element.name == 'a' else element.get('href', search_url)
            
            # Avoid duplicates
            if summary not in [item['summary'] for item in relevant_articles]:
                relevant_articles.append({'summary': summary, 'link': link})
        
        return relevant_articles
    
    # Format the output with summaries and links
    display_output = "### Summaries with Links to Search Results:\n\n"
    for keyword in keywords:
        # Encode the keyword and create the complete search URL
        encoded_keyword = urllib.parse.quote(keyword)
        search_url = (
            f"{base_url}?keyword={encoded_keyword}&dlv_OC%20CL%20Public%20Landing%20Page%203%20Column=(keyword={encoded_keyword})"
        )
        
        display_output += f"**Keyword: {keyword.capitalize()}**\n\n"
        articles = get_relevant_content(search_url)
        
        for i, article in enumerate(articles, 1):
            display_output += f"{i}. {article['summary']} "
            display_output += f"[Read More]({article['link']})\n\n"
            display_output += "---\n"  # Separator between summaries
        
        if not articles:
            display_output += f"No articles found for **{keyword}**.\n\n"

    return display_output


In [ ]:
# Usage example with keywords
base_url = "https://www.denvergov.org/Government/Agencies-Departments-Offices/Agencies-Departments-Offices-Directory"
keywords = ["Police"]
print(fetch_and_display_search_results(base_url, keywords))